In [ ]:
import os
import pandas as pd
from dotenv import load_dotenv
from sqlalchemy import create_engine
from sqlalchemy.engine import URL


## Connecting to Supabase + Loading Data

In [7]:
# Load environment variables from .env
load_dotenv()

# Fetch variables
USER = os.getenv("user")
PASSWORD = os.getenv("password")
HOST = os.getenv("host")
PORT = os.getenv("port")
DBNAME = os.getenv("dbname")

# Connect to the database
try:
    engine = create_engine(f"postgresql+psycopg2://{USER}:{PASSWORD}@{HOST}:{PORT}/{DBNAME}")
    print("Connection successful!")

except Exception as e:
    print(f"Failed to connect: {e}")

Connection successful!


## Data Cleaning and Preprocessing

In this notebook, we process our data in aita_posts.csv so that it can be fed to our logistic regression model

### Necessary Imports Below

In [16]:
import pandas as pd
import numpy as np
import spacy
from tqdm import tqdm

### Preprocessing

Here, we will begin preprocessing each reddit post. Specifically, the code below will do the following to the post title and body

- Stop word removal
- Lowercasing
- Lemminization
- Punctuation Removal
- Tokenization
- Enforce str data type

Doing this will remove irrelevant and noisy text, avoid hitting token length limits, normalize slangy/repetitive posts, and keep the BERT model efficient and focused.

Afterwards, the title and body will be combined and added a new column called "combined_text". Then, we will drop the title, selftext, and other unecessary columns to save space.

In [14]:
# Make a query to get the data from the aita_posts table
query = "SELECT id, title, body, verdict FROM aita_posts;"

# Load dataset
df = pd.read_sql(query, engine)

# print initial dataset shape
print("Initial dataset shape:", df.shape)

Initial dataset shape: (19722, 4)


In [17]:
# Load spaCy and disable unnecessary components
nlp = spacy.load("en_core_web_sm", disable=['parser', 'ner'])
nlp.max_length = 60000000

def preprocess_text(text, nlp):
    """
    Preprocesses text by lemmatizing, removing stopwords (except 'not'), and lowercasing.
    """
    # Process the text with spaCy
    doc = nlp(text.lower())
    
    # Get all stopwords except 'not'
    stop_words = nlp.Defaults.stop_words - {'not'}
    
    # Create list of tokens: lemmatized, without stopwords, and without punctuation
    tokens = [token.lemma_ for token in doc 
             if (not token.is_punct 
                 and not token.is_space
                 and token.text not in stop_words)]
    
    # Join tokens back into a string
    return ' '.join(tokens)

# Enforce strs in cols
df['body'] = df['body'].fillna('').astype(str)
df['title'] = df['title'].fillna('').astype(str)

# Add processed texts as new column
df['combined_text'] = df['title'] + ' ' + df['body']

# Apply preprocessing
tqdm.pandas(desc="Preprocessing texts")
df['combined_text'] = df['combined_text'].progress_apply(lambda x: preprocess_text(x, nlp))

# Drop the body and title columns
df.drop(columns=['body', 'title'], inplace=True)

Preprocessing texts:   3%|▎         | 591/19722 [00:16<08:48, 36.22it/s]


KeyboardInterrupt: 

After the cleaning, we save the cleaned data to Supabase

In [ ]:
# Save the preprocessed DataFrame to Supabase
try:
    df.to_sql(
        name='aita_posts_preprocessed', 
        con=engine, 
        if_exists='replace',
        index=False,
        schema='public' 
    )
    print("Successfully saved to Supabase!")
except Exception as e:
    print(f"Error saving to database: {e}")

## Logistic Regression Model

This step, we will be creating and training a Logistic Regression model as a baseline comparison to our BERT model predictor. To compare the two models, we will be comparing their accuracies, f1-scores, and individual precisions scores per classification.

Here we import all of the libraries necessary

In [19]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report
from sklearn.preprocessing import LabelEncoder, StandardScaler
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import GridSearchCV
from collections import Counter

### Dataset Preparation

To start, we will load the preprocessed dataset, vectorize the post text using TF-IDF, and split the dataset into the a training and test set

In [20]:
# Make a query to get the data from the aita_posts_preprocessed table
query = "SELECT id, combined_text, verdict FROM aita_posts_preprocessed;"

# Load the dataset
df = pd.read_sql(query, engine)

# Initialize a label encoder and apply one-hot encoding to the 'verdict' column
le = LabelEncoder()
df['verdict_encoded'] = le.fit_transform(df['verdict'])

# Split data
X = df['combined_text']
y = df['verdict_encoded']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

KeyboardInterrupt: 

Next, we need to vectorize each reddit post using TF-IDF. Once this is done, we will use SMOTE to counteract class imbalance within our data. Lastly, we will scale all features to make sure our model converges faster and prevent larger features from dominating in the training process.

In [ ]:
# Vectorize text with TF-IDF
vectorizer = TfidfVectorizer(max_features=10000, ngram_range=(1, 3), stop_words='english')
X_train_tfidf = vectorizer.fit_transform(X_train)
X_test_tfidf = vectorizer.transform(X_test)

# Apply SMOTE to balance classes
smote = SMOTE(random_state=42)
X_train_tfidf_smote, y_train_smote = smote.fit_resample(X_train_tfidf, y_train)
print("Training class distribution after SMOTE:", Counter(y_train_smote))

# Scale features
scaler = StandardScaler(with_mean=False)  # For sparse matrices
X_train_tfidf_smote_scaled = scaler.fit_transform(X_train_tfidf_smote)
X_test_tfidf_scaled = scaler.transform(X_test_tfidf)

After data preparation, we will tune the model and list the best parameters for training.

In [ ]:
# Tune model with GridSearchCV
param_grid = {'C': [0.001, 0.01, 0.1, 1, 10, 100, 1000], 'solver': ['lbfgs', 'liblinear']}
grid = GridSearchCV(LogisticRegression(class_weight='balanced', max_iter=5000, tol=0.001),  # Increased max_iter, tol
                    param_grid, cv=5, scoring='f1_macro')
grid.fit(X_train_tfidf_smote_scaled, y_train_smote)
print("Best parameters:", grid.best_params_)
print("Best cross-validation F1-macro score:", grid.best_score_)

Best parameters: {'C': 100, 'solver': 'lbfgs'}
Best cross-validation F1-macro score: 0.9219754085549277


Now, we will begin training the model using the training set

In [ ]:
# Train model with best parameters
model = grid.best_estimator_
model.fit(X_train_tfidf_smote, y_train_smote)

LogisticRegression(C=100, class_weight='balanced', max_iter=5000, tol=0.001)

After training the model, we can use it on the testing data and compute test results.

In [ ]:
y_pred = model.predict(X_test_tfidf)
print("Accuracy:", accuracy_score(y_test, y_pred))
print("Prediction distribution:", Counter(y_pred))
print(classification_report(y_test, y_pred, target_names=le.classes_))

Accuracy: 0.544
Prediction distribution: Counter({np.int64(3): 1406, np.int64(0): 465, np.int64(2): 72, np.int64(1): 57})
                  precision    recall  f1-score   support

         asshole       0.26      0.27      0.26       447
  everyone sucks       0.09      0.05      0.06       102
no assholes here       0.07      0.05      0.05       111
 not the asshole       0.68      0.72      0.70      1340

        accuracy                           0.54      2000
       macro avg       0.27      0.27      0.27      2000
    weighted avg       0.52      0.54      0.53      2000

Top features for asshole: [(np.float64(11.898845336043651), 'edit'), (np.float64(9.015123299032348), 'generally'), (np.float64(8.979314587484486), 'wibta'), (np.float64(8.14491262413867), 'wouldn'), (np.float64(7.8954154023753365), 'gay'), (np.float64(7.831789179331353), 'fit'), (np.float64(7.754107082748809), 'slowly'), (np.float64(7.6861639602708465), 'hoping'), (np.float64(7.489347113492271), 'attend'), (